In [23]:
import numpy as np 
import torch 
import torchvision 
import torch.nn as nn 
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms, models 

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
#define Hyperparameters 
in_channels = 3 
alpha = 1e-3
batch_size = 128
epochs = 10 
num_classes = 40 

In [26]:
#Function Definition for training 
#Inputs: model, data loaders for train + validation, loss function, optimizer, # epochs 
#GPU/CPU 
#outputs: none 
#Does: Trains model against new dataset, updates weights and prints training/validation 
#loss and accuracy 
def train_model(model, data_loaders, criterion, optimizer, epochs, device):
  
  for epoch in range(epochs):
    
    #State current epoch 
    print("Epoch %d / %d" % (epoch, epochs-1))
    print("-"*10)

    #either train or evaluate based on dataset 
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else: 
        model.eval()
      
      running_loss = 0.0 
      correct = 0 

      #Load data in batches 
      for x_batch, y_batch in data_loaders[phase]: 
        x_var = x_batch.to(device)
        y_var = y_batch.to(device)

        optimizer.zero_grad()
        
        with torch.set_grad_enabled(phase=='train'):
          outputs = model(x_var)
          loss = criterion(outputs, y_var)

          _, preds = torch.max(outputs,1)

          if phase == 'train':
            loss.backward()
            optimizer.step()
        
        running_loss +=loss.item() * x_var.size(0)
        correct += torch.sum(preds == y_var.data)
    
    epochLoss = running_loss / len(data_loaders[phase].dataset)
    epochAcc = 100 * (correct.double() / len(data_loaders[phase].dataset))

    print('Phase: {}, current Epoch loss: {:.4f}, accuracy: {:.2f} %'.format(phase, epochLoss, epochAcc))

In [36]:
def test_model(model, data_loader): 
  correct = 0
  for batch_idx, (x_batch, y_batch) in enumerate(data_loader):
    x_test = x_batch.to(device)
    y_test = y_batch.to(device)
    y_pred = model(x_test)

    _, preds = torch.max(y_pred, 1)
    correct += torch.sum(preds == y_test.data)

  accuracy = 100 * (correct.double() / len(data_loader.dataset))

  print("Test Set Accuracy: {:.2f}".format(accuracy))

In [28]:
def parameter_gradients(model, detach):
  if detach:
    for param in model.parameters():
      param.requires_grad = False

In [29]:
train_val_set = torchvision.datasets.GTSRB(root='./data', split = "train", 
                                           transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),
                                                           transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))]), 
                                           download = True)

train_set, val_set = torch.utils.data.random_split(train_val_set, [20000,6640])

train_loader = torch.utils.data.DataLoader(dataset = train_set,
                                           batch_size = batch_size,
                                           shuffle = True)

val_loader = torch.utils.data.DataLoader(dataset = val_set,
                                           batch_size = batch_size,
                                           shuffle = True)

data_loaders = {}

data_loaders['train'] = train_loader 
data_loaders['val'] = val_loader

In [30]:
#Import pretrained resnet weights 
model = models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
#You're able to verify what the minimum required input size is for a model by simply googling 
#32x32 is the min for ResNet50, so at first I tried transforming input from GTSRB to 32x32

In [31]:
#Swap the final fully connected layer in the neural net to classify over current 
#GTSRB dataset
parameter_gradients(model, True)
model.fc = nn.Linear(in_features = 2048, out_features = 43, bias = True)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
update_params = []
for name, param in model.named_parameters():
  if param.requires_grad:
    update_params.append(param)
    print('\n', name)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = alpha)


 fc.weight

 fc.bias


In [33]:
train_model(model, data_loaders, criterion, optimizer, epochs, device)

Epoch 0 / 9
----------
Phase: val, current Epoch loss: 1.5321, accuracy: 67.27 %
Epoch 1 / 9
----------
Phase: val, current Epoch loss: 1.0527, accuracy: 77.12 %
Epoch 2 / 9
----------
Phase: val, current Epoch loss: 0.8677, accuracy: 81.17 %
Epoch 3 / 9
----------
Phase: val, current Epoch loss: 0.7110, accuracy: 84.82 %
Epoch 4 / 9
----------
Phase: val, current Epoch loss: 0.6155, accuracy: 86.20 %
Epoch 5 / 9
----------
Phase: val, current Epoch loss: 0.5583, accuracy: 87.97 %
Epoch 6 / 9
----------
Phase: val, current Epoch loss: 0.5260, accuracy: 88.15 %
Epoch 7 / 9
----------
Phase: val, current Epoch loss: 0.4796, accuracy: 89.05 %
Epoch 8 / 9
----------
Phase: val, current Epoch loss: 0.4428, accuracy: 89.55 %
Epoch 9 / 9
----------
Phase: val, current Epoch loss: 0.4219, accuracy: 89.79 %


In [34]:
test_dataset = torchvision.datasets.GTSRB(root='./data', split = "test", 
                                           transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),
                                                           transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))]), 
                                           download = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

In [37]:
#Now that the model is trained and validated (90% accuracy), we can test it against 
#our test set 
test_model(model, test_loader)

Test Set Accuracy: 67.02
